# Security

* This notebook uses Application Default Credentials (ADC) for BigQuery authentication.
* Authenticate with gcloud before running: `gcloud auth application-default login`
* No API key files needed - authentication is handled through your gcloud session.

# Tools

In [1]:
# Package and library installation
packages_needed = c("tidyverse", "knitr") # comma delimited vector of package names
packages_installed = packages_needed %in% rownames(installed.packages())

if (any(! packages_installed))
  install.packages(packages_needed[! packages_installed])
for (i in 1:length(packages_needed)) {
  library(packages_needed[i], character.only = T)
}

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


* Using Application Default Credentials - no manual key file needed if you've run `gcloud auth application-default login`

In [2]:
install.packages("bigrquery")
library(bigrquery)

also installing the dependency ‘nanoparquet’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



# Source

## Database Connection

In [3]:
# BigQuery Authentication using Application Default Credentials
# This will use your gcloud authentication automatically
# Run this first in terminal: gcloud auth application-default login
bq_auth()

In [4]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [5]:
billing <- bq_test_project()

### vegetation_point_intercept_gridVeg

In [6]:
con_point_intercept <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "vegetation_point_intercept_gridVeg",
  billing = billing
)

In [7]:
dbListTables(con_point_intercept)

[1] "gridVeg_additional_species"         "gridVeg_ground_cover_metadata"     
[3] "gridVeg_image_metadata"             "gridVeg_image_metadata_copy"       
[5] "gridVeg_point_intercept_ground"     "gridVeg_point_intercept_vegetation"
[7] "gridVeg_shrub_tree"                 "gridVeg_survey_metadata"

In [8]:
ground_point_intercept_sql <-
"
  SELECT
    survey_ID,
    grid_point,
    intercept_ground_code,
    COUNT(intercept_ground_code) / 2 AS intercepts_pct
  FROM
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_point_intercept_ground`
  GROUP BY
    survey_ID, grid_point, intercept_ground_code
"
bq_ground_point_intercept <- bq_project_query(billing, ground_point_intercept_sql)
tb_ground_point_intercept <- bq_table_download(bq_ground_point_intercept)
# Drop NA rows. These only occur in intercept_ground_code, are always associated with
# zero values in intercepts_pct, and will be expanded to true values later using complete()
# df_ground_point_intercept <- as.data.frame(tb_ground_point_intercept) %>% drop_na() %>% glimpse() ## LINE COMMENTED OUT; LINE BELOW ADDED; BL; 2021-01-22
df_ground_point_intercept <- as.data.frame(tb_ground_point_intercept) %>% filter(intercept_ground_code != "NA") %>% glimpse()

Rows: 10,357
Columns: 4
$ survey_ID             <chr> "D4CB77CE-87D8-481F-90CB-EC627082C96F", "6654E7B…
$ grid_point            <int> 558, 190, 119, 98, 112, 165, 549, 62, 30, 561, 1…
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BV", "G", "L", "L", "L"…
$ intercepts_pct        <dbl> 22.5, 23.0, 28.0, 21.5, 32.0, 18.0, 33.5, 48.0, …


### gridVeg_survey_metadata

In [9]:
survey_metadata_sql <-
"
  SELECT
    survey_ID,
    year,
    survey_sequence
  FROM
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata`
"
bq_survey_metadata <- bq_project_query(billing, survey_metadata_sql)
tb_survey_metadata <- bq_table_download(bq_survey_metadata)
df_survey_metadata <- as.data.frame(tb_survey_metadata) %>% glimpse()

Rows: 1,723
Columns: 3
$ survey_ID       <chr> "138", "139", "135", "134", "137", "136", "141", "143"…
$ year            <int> 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, …
$ survey_sequence <chr> "2010", "2010", "2010", "2010", "2010", "2010", "2010"…


### ground_cover_metadata

In [10]:
sql_ground_meta <-
"
SELECT
  intercept_ground_code,
  ground_group
FROM
  `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_ground_cover_metadata`
"
bq_ground_meta <- bq_project_query(billing, sql_ground_meta)
tb_ground_meta <- bq_table_download(bq_ground_meta)
df_ground_meta <- as.data.frame(tb_ground_meta) %>% arrange(intercept_ground_code)

df_ground_meta %>% kable(format = "pandoc")



intercept_ground_code   ground_group 
----------------------  -------------
BG                      inorganic    
BV                      vas_plant    
G                       inorganic    
L                       litter       
LIC                     lichen       
M                       nonvas_plant 
M/L                     nonvas_plant 
OTHER                   inorganic    
R                       inorganic    
SC                      dung         
SD                      dung         
SE                      dung         
SH                      dung         
SU                      dung         
WDL                     woody        
WDS                     woody        
WDSTUMP                 woody        
WDT                     woody        

# Wrangle

In [11]:
# show unique intercept ground codes
df_ground_point_intercept %>%
  distinct(intercept_ground_code) %>%
  arrange(intercept_ground_code)

intercept_ground_code
<chr>
BG
BV
G
L
LIC
M
M/L
OTHER
R


## Complete intercept_ground_code
- Complete the data frame for `intercept_ground_code` and fill null cells with 0
- As of 2020-08-24: with 18 ground cover codes and 1244 survey IDs, we should end up with 22,392 rows after completeing the data frame
- We should also end up with 1244 distinct combinations of survey_ID and grid_point
- What should be expected for 2025 updates?

In [12]:
df_ground_point_intercept_fill <-
  df_ground_point_intercept %>%
  complete(intercept_ground_code, nesting(survey_ID, grid_point), fill= list(intercepts_pct = 0)) %>%
  glimpse()

df_ground_point_intercept_fill %>%
  distinct(survey_ID, grid_point) %>%
  count()

Rows: 26,910
Columns: 4
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG", …
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "0133805…
$ grid_point            <int> 285, 505, 401, 23, 126, 306, 305, 323, 142, 511,…
$ intercepts_pct        <dbl> 15.5, 14.5, 16.5, 16.5, 35.0, 20.5, 1.5, 0.0, 7.…


n
<int>
1495


## Join datasets

In [13]:
df_gridVeg_groundCover_intercepts_join <-
  df_ground_point_intercept_fill %>%
  left_join(df_survey_metadata, by = "survey_ID") %>%
  left_join(df_ground_meta, by = "intercept_ground_code") %>%
  select(survey_ID, year, survey_sequence, grid_point, intercept_ground_code, ground_group, intercepts_pct) %>%
  glimpse()

Rows: 26,910
Columns: 7
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "0133805…
$ year                  <int> 2016, 2016, 2017, 2016, 2025, 2023, 2021, 2015, …
$ survey_sequence       <chr> "2016", "2016", "2017", "2016", "2025", "2023", …
$ grid_point            <int> 285, 505, 401, 23, 126, 306, 305, 323, 142, 511,…
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG", …
$ ground_group          <chr> "inorganic", "inorganic", "inorganic", "inorgani…
$ intercepts_pct        <dbl> 15.5, 14.5, 16.5, 16.5, 35.0, 20.5, 1.5, 0.0, 7.…


## Filter

In [16]:
# df_gridVeg_groundCover_intercepts
# df_gridVeg_groundCover_intercepts_join
df_gridVeg_groundCover_intercepts <- df_gridVeg_groundCover_intercepts_join %>%
  filter(year > 2022 & grid_point != 586) %>%
  glimpse()

Rows: 1,944
Columns: 7
$ survey_ID             <chr> "01C59CA8-FEA9-4CAC-AB3E-1B08D747F8C5", "01E1461…
$ year                  <int> 2025, 2023, 2023, 2025, 2023, 2024, 2024, 2025, …
$ survey_sequence       <chr> "2025", "2023", "2023", "2025", "2023", "2024", …
$ grid_point            <int> 126, 306, 227, 28, 305, 319, 320, 466, 119, 55, …
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG", …
$ ground_group          <chr> "inorganic", "inorganic", "inorganic", "inorgani…
$ intercepts_pct        <dbl> 35.0, 20.5, 5.0, 45.0, 1.0, 3.5, 2.0, 43.0, 28.0…


## QC
* No rows contain missing data, and this makes sense since `complete()` was used to fill response variables.
* Numeric variables appear correct in the summary

In [17]:
df_gridVeg_groundCover_intercepts[which(!complete.cases(df_gridVeg_groundCover_intercepts)), ]

survey_ID,year,survey_sequence,grid_point,intercept_ground_code,ground_group,intercepts_pct
<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>


In [18]:
summary(df_gridVeg_groundCover_intercepts)

  survey_ID              year      survey_sequence      grid_point    
 Length:1944        Min.   :2023   Length:1944        Min.   :  2.00  
 Class :character   1st Qu.:2023   Class :character   1st Qu.: 72.25  
 Mode  :character   Median :2024   Mode  :character   Median :134.50  
                    Mean   :2024                      Mean   :202.62  
                    3rd Qu.:2025                      3rd Qu.:309.25  
                    Max.   :2025                      Max.   :582.00  
 intercept_ground_code ground_group       intercepts_pct  
 Length:1944           Length:1944        Min.   : 0.000  
 Class :character      Class :character   1st Qu.: 0.000  
 Mode  :character      Mode  :character   Median : 0.000  
                                          Mean   : 5.554  
                                          3rd Qu.: 3.000  
                                          Max.   :95.500  

In [19]:
df_gridVeg_groundCover_intercepts %>%
  group_by(year) %>%
  summarize(count_distinct = n_distinct(survey_ID))

year,count_distinct
<int>,<int>
2023,36
2024,33
2025,39


# Output

In [ ]:
# Output 2022-08-17 esamsoe
# write_csv(df_gridVeg_groundCover_intercepts, file = "gridVeg_groundCover_intercepts_WRANGLE.csv")

# Output 2025-11-04 esamsoe
# write_csv(df_gridVeg_groundCover_intercepts, file = "../data/processed/gridVeg_groundCover_intercepts_WRANGLE-251104.csv")
